In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [3]:
mu = x_train.mean(dim =0) # 평균 및 표준편차는 각 원소에 적용될거기 때문에
                          # 0차원에 적용한다
sigma = x_train.std(dim=0)


In [4]:
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [5]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self, x):
        return self.linear(x)
    
model = MultivariateLinearRegressionModel()

In [6]:
optimizer = optim.SGD(model.parameters(), lr = 1e-1)

def train(model, optimizer, x_train, y_train):
    epochs = 20
    for epoch in range(epochs+1):
        prediction = model(x_train)
        print(prediction)
        loss = F.mse_loss(prediction, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("loss:{:.5f}".format(loss))

train(model, optimizer, norm_x_train, y_train)
# train(model, optimizer, x_train, y_train)
# 위의 x_train 인자 같은 경우 데이터의 값이 너무 크기때문에 
# 최적화 범위를 벗어날 수 있다.
# 그래서 위와 같은 식을 사용하며, 한편 loss값이 지나치게 작아지며
# overfitting현상이 발생하는것을 방지하기 위해 w, b값에도
# 일정 오차를 부여해준다 ( 아래부터 작성 )


In [7]:
def train_with_regularization(model, optimizer, x_train, y_train):
    epochs = 20
    for epoch in range(epochs+1):
        prediction = model(x_train)
        # print(prediction)
        loss = F.mse_loss(prediction, y_train)
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        loss += l2_reg
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("loss:{:.5f}".format(loss))

In [28]:
train_with_regularization(model, optimizer, norm_x_train, y_train)


loss:184.48463
loss:184.48462
loss:184.48459
loss:184.48459
loss:184.48456
loss:184.48454
loss:184.48453
loss:184.48451
loss:184.48450
loss:184.48448
loss:184.48447
loss:184.48445
loss:184.48444
loss:184.48444
loss:184.48442
loss:184.48441
loss:184.48441
loss:184.48439
loss:184.48438
loss:184.48438
loss:184.48434


# norm test

In [9]:
a = torch.arange(9, dtype = torch.float)-4
a

tensor([-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])

In [10]:
b = a.reshape((3,3))
b

tensor([[-4., -3., -2.],
        [-1.,  0.,  1.],
        [ 2.,  3.,  4.]])

In [11]:
torch.norm(a)

tensor(7.7460)

In [12]:
torch.norm(b)

tensor(7.7460)

In [13]:
torch.norm(a, float('inf'))

tensor(4.)

In [14]:
torch.norm(b, float('inf'))

tensor(4.)

In [15]:
c = torch.tensor([[1,2,3],[-1,1,4]], dtype = torch.float)
torch.norm(c,dim=0)

tensor([1.4142, 2.2361, 5.0000])

In [16]:
torch.norm(c,dim=1)

tensor([3.7417, 4.2426])

In [17]:
torch.norm(c,p=1,dim=1) # 맨헤튼 놈 (절대값 합)

tensor([6., 6.])

In [18]:
d = torch.arange(8, dtype=torch.float).reshape(2,2,2)

d

tensor([[[0., 1.],
         [2., 3.]],

        [[4., 5.],
         [6., 7.]]])

In [19]:
torch.norm(d, dim=(1,2))

tensor([ 3.7417, 11.2250])

In [20]:
a = torch.FloatTensor([0.1160,0.1387,-0.3866])
b = torch.FloatTensor([-0.2739])
torch.norm(b)

tensor(0.2739)

In [21]:
0.2739+0.4268

0.7007